In [7]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from script.feature_extractor import FeatureExtractor
from script.classifier import Classifier
from script.signal_manager import SignalMgr
from script.filter import Filter
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from script.model import ModelVersionFeatureConfig
%matplotlib inline

feaExtractor = FeatureExtractor()
# classifier = Classifier()
sigMgr = SignalMgr()

# norm_signal = sigMgr.normalize_signals(signals)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from script.signal_manager import SignalMgr 
from script.model import ModelVersionFeatureConfig
from script.data_reader import DataReader
from sklearn.metrics import classification_report

class Eval(object):
    """
    evaluation class used to compare model & features on test suites
    """
    def __init__(self):
        self.features = []
        self.model_path = []
        self.dreader = DataReader()
        self.request_param = dict()
        self.request_param['skip_row'] = [1]
        self.sigMgr = SignalMgr()
        pass
    
    def eval(self, test_suite_dirs, test_suite_index=None, model_version = ''):
        """
        load model from model_version or model_path (absolute path)
        """
        # initialize classifier
        if not ModelVersionFeatureConfig.has_key(model_version):
            print "model_version: %s is not exist" % model_version
            return False
        
        self.request_param['model_version'] = model_version
        # read target evaluation datas
        if test_suite_index is None:
            case_labels = list()
            for test_suite_sub_dir in test_suite_dirs:
                case_dir = self.dreader.search_label_files(test_suite_sub_dir)
                case_labels.extend(case_dir)

            #create data frames
            eval_df = self.dreader.create_index(case_labels)
        else:
            eval_df = pd.read_csv(test_suite_index)
#         print (eval_df)
        sys_eval_result = list()
        for index, row in eval_df.iterrows():
            case_path = row['case_path']
            ret = self.sigMgr.process(case_path, self.request_param)
            sys_eval_result.append(int(ret['stat']))
        
        eval_df['sys_result'] = sys_eval_result
        print(classification_report(eval_df['expect_result'], eval_df['sys_result']))
        return eval_df

In [9]:
evalMgr = Eval()

In [10]:
from script.model import ModelVersionFeatureConfig
print (len(ModelVersionFeatureConfig['gbdt_0830BA_SKEW_LS']['features']))

15


In [13]:
df_0830_predict = evalMgr.eval(['/Volumes/workspace/projects/signal_classification/data/特殊次品样本/斜角_轻微.20190515/'], '/Users/changkong/project/signal_classification/data/eval_skew.csv', model_version='gbdt_0830BA_SKEW_LS')

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.82      0.90        76

   micro avg       0.82      0.82      0.82        76
   macro avg       0.50      0.41      0.45        76
weighted avg       1.00      0.82      0.90        76



In [6]:
df_0830_1005_predict = evalMgr.eval(['/Users/changkong/ML/Signal Classification/testData/0830_1005_Data.20180614'], model_version='gbdt_0830_BA')

/Users/changkong/anaconda3/envs/tpy2/lib/python2.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/Users/changkong/anaconda3/envs/tpy2/lib/python2.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.19.1 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/changkong/anaconda3/envs/tpy2/lib/python2.7/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator GradientBoostingClassifier from version 0.19.1 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


/Users/changkong/ML/Signal Classification/testData/0830_1005_Data.20180614/result.csv 1700 1700


KeyboardInterrupt: 

In [ ]:
error_result = df_0830_1005_predict[df_0830_1005_predict.sys_result != df_0830_1005_predict.expect_result]

In [ ]:
tp = error_result[error_result.expect_result == 1].reset_index(drop=True)

In [ ]:
signal_fpath = tp[tp.reason == 5]['case_path'].reset_index(drop=True)[16]
df_signals = pd.read_csv(signal_fpath, skiprows=1, header=None, names=['sig'])
df_signals[0:500].plot()